In [2]:
import pandas as pd
import random

# Define recipe database (added Vegetable Soup)
recipes = [
    {"Recipe ID": "R101", "Recipe Name": "Paneer Butter Masala", "Spice Level": "Medium", "Cooking Time": 30, "Ingredients": "Paneer, Tomato, Cream"},
    {"Recipe ID": "R102", "Recipe Name": "Dal Tadka", "Spice Level": "Mild", "Cooking Time": 25, "Ingredients": "Lentils, Garlic, Ghee"},
    {"Recipe ID": "R103", "Recipe Name": "Chicken Curry", "Spice Level": "Medium", "Cooking Time": 40, "Ingredients": "Chicken, Onion, Spices"},
    {"Recipe ID": "R104", "Recipe Name": "Aloo Gobi", "Spice Level": "Mild", "Cooking Time": 35, "Ingredients": "Potato, Cauliflower, Turmeric"},
    {"Recipe ID": "R105", "Recipe Name": "Mutton Rogan Josh", "Spice Level": "Spicy", "Cooking Time": 45, "Ingredients": "Mutton, Yogurt, Kashmiri Chili"},
    {"Recipe ID": "R106", "Recipe Name": "Sambar", "Spice Level": "Mild", "Cooking Time": 30, "Ingredients": "Lentils, Tamarind, Drumstick"},
    {"Recipe ID": "R107", "Recipe Name": "Butter Chicken", "Spice Level": "Medium", "Cooking Time": 35, "Ingredients": "Chicken, Tomato, Cream"},
    {"Recipe ID": "R108", "Recipe Name": "Chana Masala", "Spice Level": "Spicy", "Cooking Time": 30, "Ingredients": "Chickpeas, Onion, Tomato"},
    {"Recipe ID": "R109", "Recipe Name": "Palak Paneer", "Spice Level": "Medium", "Cooking Time": 35, "Ingredients": "Spinach, Paneer, Cream"},
    {"Recipe ID": "R110", "Recipe Name": "Biryani", "Spice Level": "Spicy", "Cooking Time": 50, "Ingredients": "Rice, Chicken/Mutton, Saffron"},
    {"Recipe ID": "R111", "Recipe Name": "Vegetable Soup", "Spice Level": "Mild", "Cooking Time": 20, "Ingredients": "Carrot, Tomato, Water, Salt, Pepper"},
]

# Possible feedback responses
feedback_options = ["Perfect!", "Too spicy!", "Reduce salt", "Add more turmeric", "Too oily", "Loved it!", "Needs more seasoning"]

# Generate random user preferences
users = ["U1", "U2", "U3", "U4", "U5"]
data = []

for _ in range(60):  # Generate 60 random user-recipe interactions
    user = random.choice(users)
    recipe = random.choice(recipes)
    rating = random.randint(1, 5)  # Random rating between 1-5
    feedback = random.choice(feedback_options)

    data.append({
        "User ID": user,
        "Recipe ID": recipe["Recipe ID"],
        "Recipe Name": recipe["Recipe Name"],
        "Spice Level": recipe["Spice Level"],
        "Cooking Time (mins)": recipe["Cooking Time"],
        "Ingredients": recipe["Ingredients"],
        "User Rating": rating,
        "Feedback": feedback
    })

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("recipe_recommendation_dataset.csv", index=False)

print("Dataset updated and saved as recipe_recommendation_dataset.csv")

Dataset updated and saved as recipe_recommendation_dataset.csv


In [3]:
import surprise
print(surprise.__version__)  # Should print the installed version

ModuleNotFoundError: No module named 'surprise'

In [4]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505213 sha256=f4a09707d6f1233b55e191364032ba10a8935e1e903182c7d549ec59ffbb67a5
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [5]:
import surprise
print(surprise.__version__)  # Should print the installed version


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [6]:
pip install scikit-surprise pandas

In [7]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load dataset from CSV
file_path = "recipe_recommendation_dataset.csv"
df = pd.read_csv(file_path)

# Ensure the dataset has the required columns
if not {"user_id", "recipe_id", "rating"}.issubset(df.columns):
    raise ValueError("Dataset must contain 'user_id', 'recipe_id', and 'rating' columns.")

# Convert dataset into Surprise format
reader = Reader(rating_scale=(1, 5))  # Assuming ratings are between 1 and 5
recipe_data = Dataset.load_from_df(df[["user_id", "recipe_id", "rating"]], reader)

# Train-Test Split
trainset, testset = train_test_split(recipe_data, test_size=0.2)

# Train the model using SVD
model = SVD()
model.fit(trainset)

# Make predictions and evaluate model performance
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse:.2f}")

# Function to get top N recipe recommendations for a user
def get_recommendations(user_id, df, model, top_n=5):
    all_recipes = df["recipe_id"].unique()
    rated_recipes = df[df["user_id"] == user_id]["recipe_id"].values
    unrated_recipes = [recipe for recipe in all_recipes if recipe not in rated_recipes]

    # Predict ratings for unrated recipes
    predictions = [(recipe, model.predict(user_id, recipe).est) for recipe in unrated_recipes]
    predictions.sort(key=lambda x: x[1], reverse=True)  # Sort by predicted rating

    return predictions[:top_n]

# Example: Recommend top 5 recipes for a given user (e.g., User ID 1)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [8]:
!pip install "numpy<2"
!pip uninstall -y scikit-surprise
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Found existing installation: scikit-surprise 1.1.4
Uninstalling scikit-surprise-1.1.4:
  Successfully uninstalled scikit-surprise-1.1.4
  Using cached scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl


In [2]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Step 1: Load the Dataset
file_path = "recipe_recommendation_dataset.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Step 2: Define the Data Format for `surprise`
reader = Reader(rating_scale=(1, 5))  # Assuming ratings are from 1 to 5
data = Dataset.load_from_df(df[['User ID', 'Recipe ID', 'User Rating']], reader)

# Step 3: Train-Test Split
trainset, testset = train_test_split(data, test_size=0.2)

# Step 4: Train an SVD Model
model = SVD()
model.fit(trainset)

# Step 5: Make Predictions
predictions = model.test(testset)

# Step 6: Evaluate the Model
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse:.4f}")

# Step 7: Recommend Recipes for a Specific User
def recommend_recipes(user_id, df, model, top_n=5):
    unique_recipes = df["Recipe ID"].unique()
    user_rated_recipes = df[df["User ID"] == user_id]["Recipe ID"].tolist()

    # Predict ratings for recipes not yet rated by the user
    predictions = [
        (recipe_id, model.predict(user_id, recipe_id).est)
        for recipe_id in unique_recipes if recipe_id not in user_rated_recipes
    ]

    # Sort by predicted rating
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]

    # Get Recipe Names
    recommended_df = pd.DataFrame(recommendations, columns=["Recipe ID", "Predicted Rating"])
    recommended_df = recommended_df.merge(df[["Recipe ID", "Recipe Name"]].drop_duplicates(), on="Recipe ID", how="left")

    return recommended_df

# Example: Get Top 5 Recommendations for User ID 101
user_id = 101  # Change to any user ID you want
recommended_recipes = recommend_recipes(user_id, df, model, top_n=5)
print("\nRecommended Recipes:")
print(recommended_recipes)

RMSE: 1.8562
RMSE: 1.8562

Recommended Recipes:
  Recipe ID  Predicted Rating           Recipe Name
0      R106          2.859978                Sambar
1      R102          2.811201             Dal Tadka
2      R101          2.659933  Paneer Butter Masala
3      R103          2.626482         Chicken Curry
4      R109          2.616686          Palak Paneer
